In [1]:
#Implementation found in https://github.com/dengjunquan/DoA-Estimation-MUSIC-ESPRIT/blob/master/DoAEstimation.py
# https://dengjunquan.github.io/posts/2018/08/DoAEstimation_Python/

import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as LA
import scipy.signal as ss
import operator

# Functions
def array_response_vector(array,theta):
    N = array.shape
    v = np.exp(1j*2*np.pi*array*np.sin(theta))
    
    print("The array is ")
    print(array)
    print("The theta is ")
    print(theta)
    print("The v is ")
    print(v)
    

    return v/np.sqrt(N)

def music(CovMat,L,N,array,Angles):
    # CovMat is the signal covariance matrix, L is the number of sources, N is the number of antennas
    # array holds the positions of antenna elements
    # Angles are the grid of directions in the azimuth angular domain
    _,V = LA.eig(CovMat)
    Qn  = V[:,L:N]
    numAngles = Angles.size
    pspectrum = np.zeros(numAngles)
    for i in range(numAngles):
        av = array_response_vector(array,Angles[i])
        pspectrum[i] = 1/LA.norm((Qn.conj().transpose()@av))
    psindB    = np.log10(10*pspectrum/pspectrum.min())
    DoAsMUSIC,_= ss.find_peaks(psindB,height=1.35, distance=1.5)
    return DoAsMUSIC,pspectrum

def esprit(CovMat,L,N):
    # CovMat is the signal covariance matrix, L is the number of sources, N is the number of antennas
    _,U = LA.eig(CovMat)
    S = U[:,0:L]
    Phi = LA.pinv(S[0:N-1]) @ S[1:N] # the original array is divided into two subarrays [0,1,...,N-2] and [1,2,...,N-1]
    eigs,_ = LA.eig(Phi)
    DoAsESPRIT = np.arcsin(np.angle(eigs)/np.pi)
    return DoAsESPRIT
#=============================================================


lamda = 1 # wavelength
kappa = np.pi/lamda # wave number
L = 5  # number of sources
N = 32  # number of ULA elements 
snr = 10 # signal to noise ratio


array = np.linspace(0,(N-1)/2,N)
print(array.shape)

plt.figure()
plt.subplot(221)
plt.plot(array,np.zeros(N),'^')
plt.title('Uniform Linear Array')
plt.legend(['Antenna'])

#Transform our thetas and angles in radians so it works easier 
Thetas = np.pi*(np.random.rand(L)-1/2)   # random source directions
Alphas = np.random.randn(L) + np.random.randn(L)*1j # random source powers
Alphas = np.sqrt(1/2)*Alphas

#print(Thetas)
#print(Alphas)
#print(array)

# A CHANGER Array_response_vector dans array surement mettre mes réponses de lego et Thetas
# theta est en radians mes response sont en degrés donc quelque chose à changer 
h = np.zeros(N)
print(array_response_vector(array,Thetas[0]).shape)
for i in range(L):
    #print("The response is ")
    #print(array_response_vector(array,Thetas[i]))
    h = h + Alphas[i]*array_response_vector(array,Thetas[i])

# ADAPTER avec anglesnf donc des angles en degrés 
Angles = np.linspace(-np.pi/2,np.pi/2,360)
numAngles = Angles.size
#print(Angles)

# A CHANGER Array_response_vector dans array surement mettre mes réponses de lego et Thetas

hv = np.zeros(numAngles)
for j in range(numAngles):
    av = array_response_vector(array,Angles[j])
    hv[j] = np.abs(np.inner(h,av.conj()))

powers = np.zeros(L)
for j in range(L):
    av = array_response_vector(array,Thetas[j])
    powers[j] = np.abs(np.inner(h,av.conj()))

plt.subplot(222)
plt.plot(Angles,hv)
plt.plot(Thetas,powers,'*')
plt.title('Correlation')
plt.legend(['Correlation power','Actual DoAs'])
numrealization = 100
H = np.zeros((N,numrealization)) + 1j*np.zeros((N,numrealization))
#print(H.shape)
for iter in range(numrealization):
    htmp = np.zeros(N)
    for i in range(L):
        pha = np.exp(1j*2*np.pi*np.random.rand(1))
        htmp = htmp + pha*Alphas[i]*array_response_vector(array,Thetas[i])
    H[:,iter] = htmp + np.sqrt(0.5/snr)*(np.random.randn(N)+np.random.randn(N)*1j)
CovMat = H@H.conj().transpose()

# MUSIC algorithm
DoAsMUSIC, psindB = music(CovMat,L,N,array,Angles)


plt.subplot(223)
plt.plot(Angles,psindB)
plt.plot(Angles[DoAsMUSIC],psindB[DoAsMUSIC],'x')
plt.title('MUSIC')
plt.legend(['pseudo spectrum','Estimated DoAs'])

# ESPRIT algorithm
DoAsESPRIT = esprit(CovMat,L,N)
plt.subplot(224)
plt.plot(Thetas,np.zeros(L),'*')
plt.plot(DoAsESPRIT,np.zeros(L),'x')
plt.title('ESPRIT')
plt.legend(['Actual DoAs','Estimated DoAs'])

print('Actual DoAs:',np.sort(Thetas),'\n')
print('MUSIC DoAs:',np.sort(Angles[DoAsMUSIC]),'\n')
print('ESPRIT DoAs:',np.sort(DoAsESPRIT),'\n')

plt.show()

(32,)
The array is 
[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5]
The theta is 
1.440919806083372
The v is 
[ 1.        +0.j         -0.99964999+0.02645574j  0.99860019-0.05289296j
 -0.99685134+0.07929316j  0.99440467-0.10563785j -0.99126189+0.13190858j
  0.98742519-0.15808698j -0.98289727+0.18415472j  0.97768129-0.21009353j
 -0.97178091+0.23588528j  0.96520025-0.2615119j  -0.95794393+0.28695546j
  0.95001701-0.31219814j -0.94142506+0.33722227j  0.93217408-0.36201033j
 -0.92227056+0.38654498j  0.91172141-0.41080904j -0.90053404+0.43478551j
  0.88871627-0.45845763j -0.87627637+0.48180881j  0.86322305-0.50482271j
 -0.84956545+0.52748322j  0.83531313-0.54977448j -0.82047607+0.57168088j
  0.80506465-0.59318708j -0.78908966+0.61427804j  0.77256228-0.63493899j
 -0.75549409+0.65515546j  0.73789704-0.6749133j  -0.71978343+0.69419868j
  0.70116596-0.71299811j -0.68205764+0.73129

 -0.75789598-0.65237542j  0.10316921+0.99466382j]
The array is 
[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5]
The theta is 
-0.8532180605153339
The v is 
[ 1.        +0.j         -0.71462003-0.69951284j  0.02136357+0.99977177j
  0.68408635-0.72940103j -0.9990872 +0.0427174j   0.74384909+0.66834761j
 -0.06405172-0.99794658j -0.6523038 +0.75795762j  0.99635045-0.08535681j
 -0.77172017-0.63596225j  0.10662293+0.99429953j  0.6193304 -0.78513047j
 -0.99179475+0.12784039j  0.79818239+0.60241586j -0.14899949-0.98883727j
 -0.58522635+0.81086998j  0.98542843-0.17009059j -0.82318744-0.5677697j
  0.19110404+0.98156979j  0.55005389-0.83512916j -0.9772631 +0.21203026j
  0.84668968+0.53208701j -0.23285971-0.97251034j -0.51387726+0.85786372j
  0.96731367-0.25358286j -0.86864619-0.49543294j  0.27419026+0.96167547j
  0.47676248-0.87903216j -0.9555983 +0.29467251j  0.88901689+0.45787441j

 -0.23745874-0.97139763j -0.65861708-0.75247827j]
The array is 
[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5]
The theta is 
-0.14439074870259505
The v is 
[ 1.        +0.j          0.89955688-0.43680364j  0.61840515-0.78585944j
  0.21302434-0.97704689j -0.23515013-0.97195906j -0.63608617-0.77161803j
 -0.90924126-0.41626954j -0.99974228+0.02270176j -0.88940883+0.4571126j
 -0.60040539+0.7996958j  -0.19078876+0.98163112j  0.25715471+0.96637025j
  0.65343933+0.75697889j  0.91845698+0.39552089j  0.99896926-0.04539182j
  0.87880236-0.47718594j  0.58209615-0.81311996j  0.16845483-0.98570937j
 -0.27902674-0.96028333j -0.67045568-0.74194958j -0.9271993 -0.37456836j
 -0.99768133+0.06805849j -0.86774291+0.49701332j -0.56348687+0.82612502j
 -0.14603408+0.98927956j  0.30075496+0.95370145j  0.68712646+0.72653784j
  0.93546371+0.35342277j  0.99587916-0.09069007j  0.8562362 -0.51658453

[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5]
The theta is 
0.5556856086433197
The v is 
[ 1.        +0.j         -0.08636735+0.99626336j -0.98508136-0.17208926j
  0.25652509-0.96653757j  0.94077058+0.33904384j -0.41902882+0.90797293j
 -0.86838976-0.49588228j  0.56902987-0.82231686j  0.77009855+0.63792493j
 -0.70205261+0.71212508j -0.6488297 -0.76093365j  0.81412802-0.58068543j
  0.50820154+0.86123818j -0.90191206+0.43191971j -0.35241002-0.9358457j
  0.9627855 -0.27026668j  0.18610355+0.98253014j -0.99493204+0.10054963j
 -0.01424426-0.99989855j  0.99739252+0.07216755j -0.15804004+0.9874327j
 -0.97009352-0.24273145j  0.32560886-0.94550456j  0.91384957+0.4060529j
 -0.48346239+0.87536513j -0.83033884-0.55725884j  0.62689073-0.77910719j
  0.72205305+0.69183769j -0.75161435+0.65960281j -0.59222317-0.80577399j
  0.85391184-0.52041768j  0.44472296+0.89566818j]
The array is 
[ 

 14.  14.5 15.  15.5]
The theta is 
1.2645129204560583
The v is 
[ 1.        +0.j         -0.98933074+0.14568696j  0.95755062-0.28826518j
 -0.90533778+0.42469224j  0.83380638-0.552057j   -0.74448277+0.66764167j
  0.639273  -0.76897986j -0.52042209+0.85390916j  0.39046614-0.92061729j
 -0.25217822+0.96768081j  0.10850919-0.99409545j  0.03747527+0.99929756j
 -0.18266006-0.98317613j  0.32394715+0.94607518j -0.45832169-0.88878638j
  0.58291632+0.81253219j -0.69507238-0.71893976j  0.79239662+0.61000622j
 -0.87281229-0.48805605j  0.93460343+0.35569148j -0.97645151-0.21573699j
  0.99746356+0.07117898j -0.99719121+0.07489788j  0.97564027-0.21937653j
 -0.93327061+0.35917401j  0.87098633-0.49130725j -0.79011649+0.61295672j
  0.69238673-0.72152659j -0.57988245+0.81470015j  0.45500435-0.89048922j
 -0.32041712+0.94727655j  0.17899266-0.98385041j]
The array is 
[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5

...

The v is 
[ 1.        +0.j          0.95874018-0.28428377j  0.83836548-0.54510854j
  0.64880917-0.76095116j  0.40571336-0.91400037j  0.12913823-0.9916266j
 -0.15809333-0.98742417j -0.43227909-0.90173987j -0.67079334-0.74164432j
 -0.85395397-0.52034855j -0.96664663-0.25611382j -0.99957197+0.02925534j
 -0.950013  +0.31221035j -0.8220593 +0.56940188j -0.62626957+0.77960658j
 -0.37880031+0.92547843j -0.10007259+0.99498014j  0.18691309+0.98237645j
  0.45847477+0.88870742j  0.69220328+0.72170259j  0.86881143+0.49514312j
  0.97372558+0.22772462j  0.99828825-0.05848563j  0.94047254-0.33986967j
  0.80504939-0.59320779j  0.60319386-0.79759461j  0.35156299-0.93616423j
  0.07092127-0.99748192j -0.21557284-0.97648776j -0.48427796-0.8749142j
 -0.71302065-0.70114304j -0.88292513-0.46951381j]
The array is 
[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5]
The theta is 
-0.08313406743482754

The array is 
[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5]
The theta is 
0.5994403809774393
The v is 
[ 1.        +0.j         -0.20026562+0.97974164j -0.91978736-0.39241713j
  0.56866919-0.82256632j  0.69201759+0.72188064j -0.84584385+0.53343058j
 -0.35323071-0.93553624j  0.98732378-0.15871909j -0.0422233 +0.9991082j
 -0.97041203-0.24145495j  0.43090363-0.90239795j  0.79782166+0.60289352j
 -0.75045613+0.66092026j -0.49724054-0.86761273j  0.9496165 -0.31341427j
  0.11688947+0.99314493j -0.99643439-0.0843713j   0.28221362-0.95935159j
  0.88339901+0.46862158j -0.63604252+0.77165401j -0.62864412-0.77769311j
  0.88783413-0.46016363j  0.27303882+0.96200302j -0.9971947 +0.07485137j
  0.12636881-0.99198333j  0.94658005+0.32246894j -0.50550368+0.86282445j
 -0.74411003-0.66805708j  0.80354299-0.59524672j  0.42226596+0.90647198j
 -0.9726737 +0.23217637j -0.03267976-0.99946587j]


 -0.32572211-0.94546555j  0.38498766+0.92292172j]
The array is 
[ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12.  12.5 13.  13.5
 14.  14.5 15.  15.5]
The theta is 
1.3782753285247695
The v is 
[ 1.        +0.j         -0.9983161 +0.05800834j  0.99327007-0.11582131j
 -0.9848789 +0.17324422j  0.97317085-0.23008368j -0.95818535+0.28614826j
  0.93997288-0.34124916j -0.91859476+0.39520079j  0.894123  -0.44782146j
 -0.86664001+0.49893396j  0.83623834-0.54836615j -0.80302039+0.59595155j
  0.76709803-0.6415299j  -0.72859223+0.68494771j  0.68763267-0.72605875j
 -0.64435731+0.76472456j  0.59891188-0.80081494j -0.55144943+0.83420833j
  0.5021298 -0.86479226j -0.45111911+0.89246375j  0.39858913-0.9171296j
 -0.34471678+0.93870674j  0.2896835 -0.9571225j  -0.23367462+0.97231485j
  0.17687877-0.98423265j -0.11948722+0.99283574j  0.06169327-0.99809516j
 -0.00369154+0.99999319j -0.05432261-0.99852344j  0.11215382+0.99369086j


Actual DoAs: [-1.3107377  -0.84931938 -0.82221974 -0.14405155  1.44091981] 

MUSIC DoAs: [-1.48328678 -0.83571615] 

ESPRIT DoAs: [-1.37461189 -1.3528557  -0.87892376 -0.83595341  0.311569  ] 



<Figure size 640x480 with 4 Axes>